In [5]:
%pip install tdqm
import requests
from bs4 import BeautifulSoup
from simplekml import Kml
from IPython.display import FileLink
import re
from urllib.parse import urljoin
from tqdm import tqdm  # progress bar

BASE_URL = "https://en.wikipedia.org"
LIST_URL = BASE_URL + "/wiki/List_of_ghost_towns_in_Texas"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_all_town_links_with_optional_status():
    """Get all town names, links, and status (if present)."""
    response = requests.get(LIST_URL, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    towns = []

    rows = soup.select("table.wikitable tbody tr")[1:]  # skip header
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 1:
            name_tag = cols[0].find("a")
            if name_tag and "href" in name_tag.attrs:
                name = name_tag.text.strip()
                url = urljoin(BASE_URL, name_tag["href"])
                status = cols[5].text.strip() if len(cols) >= 6 else "Unknown"
                towns.append((name, url, status))
    return towns

def extract_coordinates(town_url):
    """Get coordinates from the town's Wikipedia page."""
    try:
        response = requests.get(town_url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")

        geo = soup.select_one("span.geo")
        if geo:
            lat_str, lon_str = geo.text.split(";")
            return float(lat_str.strip()), float(lon_str.strip())

        geohack = soup.find("a", href=re.compile("geohack"))
        if geohack:
            match = re.search(r'([\d.]+);([\d.]+)', geohack.text)
            if match:
                return float(match.group(1)), float(match.group(2))
    except Exception:
        return None, None

    return None, None

# Get list of towns
towns = get_all_town_links_with_optional_status()
print(f"Found {len(towns)} town entries.")

# Generate KML with progress bar
kml = Kml()
count = 0

for name, url, status in tqdm(towns, desc="Processing towns", unit="town"):
    lat, lon = extract_coordinates(url)
    if lat is not None and lon is not None:
        pnt = kml.newpoint(name=name, coords=[(lon, lat)])
        pnt.description = f"{status}<br><a href='{url}' target='_blank'>Wikipedia page</a>"
        count += 1

print(f"{count} towns had coordinates.")

# Save to file
output_path = "/tmp/texas_ghost_towns_progress.kml"
kml.save(output_path)
print(f"KML saved to: {output_path}")

FileLink(output_path)


  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1323 sha256=269868574247f1236c32d223f5bd57cb2e5bbffc4d331057ded652ed5854675c
  Stored in directory: /Users/sbruce/Library/Caches/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm

[notice] A new release of pip is available: 23.0 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Found 555 town entries.


Processing towns: 100%|██████████| 555/555 [02:51<00:00,  3.23town/s]

340 towns had coordinates.
KML saved to: /tmp/texas_ghost_towns_progress.kml


/tmp/texas_ghost_towns_progress.kml